## 3rdラボへようこそ -  第1週、4日目

今日は、すぐに価値のあるものを構築します！

フォルダー `me` に、単一のファイル `linkedIn.pdf` を配置しました。これは、linkedInプロファイルのpdfダウンロードです。

あなたのものに置き換えてください！

`summary.txt`というファイルも作成しました

まだツールを使用するつもりはありません。明日ツールを追加します。

### 概要
個人のLinkedInプロフィール（PDF形式）とその要約テキスト（`summary.txt`）を用いて、  
AIチャットボット（LLM）を構築し、Webサイト上で自身になりきって質問応答できる仕組みを作る。  
さらに、回答品質を自動評価し、不適切な場合は再回答する機能も実装しています。

- セットアップとパッケージの導入
  - `dotenv`, `openai`, `pypdf`, `gradio`などのパッケージをインポート。
  - LinkedInのPDF（`me/linkedin.pdf`）を読み込み、テキスト化。
  - 要約テキスト（`me/summary.txt`）を読み込み。

- システムプロンプトの生成  
`Ed Donner`（例）になりきるためのプロンプトを作成し、その人物情報としてLinkedInテキストと要約を付与。

- チャットボットの実装
  - `chat()`関数: ユーザーからの質問と履歴をもとにOpenAI API（GPT-4o-mini）で回答を生成。
  - Gradioの`ChatInterface`でチャットUIを起動。

- 回答の自動評価
  - Pydanticで回答評価用モデル（`Evaluation`）を定義。
  - 評価用プロンプトを用意し、Gemini（Googleのモデル）で回答の品質を判定。
  - `evaluate()`関数で評価結果（許容/非許容、フィードバック）を取得。

- 再回答フロー
  - 評価で非許容の場合、フィードバックを付与して再度OpenAI APIに回答を生成させる（`rerun()`関数）。
  - `chat()`関数のロジックで、自動評価と再回答を組み合わせて返答の品質を担保。

- 追加機能（例）  
特定キーワード（例：patent）が含まれる場合、回答をピッグラテン（Pig Latin）に変換するようにプロンプトを追加。

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">パッケージの検索</h2>
            <span style="color:#00bfff;">このラボでは、簡単なUIを構築するための優れたGradioパッケージと、人気のPDFリーダーPyPDFを使用します。これらのパッケージのガイドは、ChatGPTまたはClaudeに問い合わせることで入手できます。また、すべてのOSSパッケージはリポジトリ<a href="https://pypi.org">https://pypi.org</a>で見つけることができます。
            </span>
        </td>
    </tr>
</table>

In [1]:
!pip install pypdf

In [2]:
# これらのパッケージが何をしているのかわからない場合は、いつでもChatGPTにガイドを尋ねることができます！

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [3]:
load_dotenv(override=True)

True

In [4]:
# openai と geminiの初期化
import os
openai = OpenAI()
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [5]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [6]:
print(linkedin[:100]+"...")

   
Contact
ed.donner@gmail.com
www.linkedin.com/in/eddonner
(LinkedIn)
edwarddonner.com (Personal)
...


In [7]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [8]:
name = "Ed Donner"

In [9]:
#「あなたは{name}として行動します。{name}のウェブサイト上の質問、特に{name}のキャリア、経歴、スキル、経験に関する質問に回答します。\
# あなたの責任は、ウェブサイト上でのやり取りにおいて、{name}をできる限り忠実に代表することです。質問への回答には、{name}の経歴とLinkedInプロフィールの概要が提供されます。
# ウェブサイトを訪れた潜在的な顧客や将来の雇用主に話しかけるかのように、プロフェッショナルで魅力的な対応を心がけてください。答えがわからない場合は、その旨を伝えてください。
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

# ## 概要:{summary} ## LinkedIn プロフィール:{linkedin}"
system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
# このコンテキストでは、常に {name} としてのキャラクターを維持しながらユーザーとチャットしてください。
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [10]:
# 最も簡単な、Gradio の Chat関数は、gpt-4o-miniを使用し、historyにも対応する。
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## OpenAIを使用していない人々のための特別なメモ

Groqのような一部のプロバイダーは、チャットで2番目のメッセージを送信するときにエラーを発生させる可能性があります。

これは、Gradioが履歴オブジェクトにいくつかの余分なフィールドを押し込むためです。 OpenAIは気にしません。しかし、他のいくつかのモデルは不平を言っています。

これが発生した場合、解決策は、この最初の行を上記のChat()関数に追加することです。履歴変数をクリーンアップします：

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

これを他のChat()関数のコールバック機能に将来的に追加する必要がある場合があります。

In [11]:
# キャリアアバター（経歴紹介AI）とチャットしてみる。
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


### 以下のようなことをChatで質問する
- hi there
- what is your greatest accomplishment?
- what is a challenge that you encountered and needed to overcome?

## 多くのことが起こりそうです...

1. LLMに回答の評価を依頼できる
2. 回答が評価に失敗した場合に再実行できる
3. これらを1つのワークフローにまとめる

すべてエージェントフレームワークなし！

In [12]:
# 評価の構造化出力のためにPydanticモデルを作成
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [13]:
# あなたは、質問への回答が適切かどうかを判断する評価者です。ユーザーとエージェントとの会話が表示されます。
# あなたの仕事は、エージェントの最新の回答が適切かどうかを判断することです。エージェントは{name}の役割を演じ、ウェブサイト上で{name}を代表しています。
# エージェントは、ウェブサイトにアクセスした潜在的な顧客や将来の雇用主に話しかけるかのように、プロフェッショナルで魅力的な対応をするように指示されています。
# エージェントには、{name}の概要とLinkedInの詳細という形で、{name}に関するコンテキストが提供されています。情報は次のとおりです。
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

# ## 概要:{summary} ## LinkedIn プロフィール:{linkedin}"
evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
# この文脈を踏まえて、最新の応答を評価し、応答が受け入れ可能かどうかとフィードバックを返信してください。
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [14]:
# ユーザーとエージェントの会話は次のとおりです: {history}
# ユーザーからの最新のメッセージは次のとおりです: {message}
# エージェントからの最新の応答は次のとおりです: {reply}
# 応答を評価し、適切かどうか、そしてフィードバックを返信してください。
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [15]:
# 評価結果を構造化出力する、evaluate関数は、Gradio の Chat関数から呼び出され、gemini-2.0-flashを使用し、historyにも対応する。
def evaluate(reply, message, history) -> Evaluation:
    # replyは応答、messageはプロンプト、historyはmessageを含まない。
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [16]:
# テスト：キャリアアバター（経歴紹介AI）に質問「特許をお持ちですか？」
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content
print(reply)

# キャリアアバター（経歴紹介AI）の応答を評価
# replyは応答、messageはプロンプト、「historyにmessageを含めない。」
evaluation = evaluate(reply, "do you hold a patent?", messages[:1])
print(evaluation)

Yes, I hold a patent for an invention related to an apparatus for determining role fitness while eliminating unwanted bias. This work was a collaboration with GQR, a rapidly growing recruitment firm, and it combines our expertise in AI with the recruitment industry to address challenges in hiring. If you'd like to know more about this patent or how it fits into the broader AI and recruitment landscape, feel free to ask!
is_acceptable=True feedback='This is a great response. The agent accurately states that they have a patent (which is verifiable from the linked in), and gives some background information on it. The agent then is engaging by saying "If you\'d like to know more about this patent or how it fits into the broader AI and recruitment landscape, feel free to ask!"'


In [17]:
# evaluate関数が失敗したときに再実行を行う関数で、システムプロンプトに拒否情報を追加して再実行
def rerun(reply, message, history, feedback):
    
    # 前の回答が拒否されました。返信しようとしましたが、品質管理によって返信が拒否されました。
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    # 回答の試み:{reply}
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    # 拒否理由:{feedback}
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [18]:
# Gradio の Chat関数、条件分岐と評価機能を追加。
def chat(message, history):

    # 自己紹介のオプション条件分岐
    if "patent" in message: # "patent"と言う文字が含まれていたら、返答は必ずピッグラテン語で行えと言う指示を加える。
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt

    # キャリアアバター（経歴紹介AI）
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    # 自己紹介の評価機能
    evaluation = evaluate(reply, message, history)

    # 評価機能が拒否した場合、再実行
    if evaluation.is_acceptable:
        print("渡された評価 - 返信を返します")
    else:
        print("評価の失敗 - 再試行")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)

    # 自己紹介の結果
    return reply

In [19]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


渡された評価 - 返信を返します
評価の失敗 - 再試行
The response is not acceptable. The agent is responding in Pig Latin, which is not professional. The agent should be responding in a professional and engaging tone, as if talking to a potential client or future employer.


### 以下のようなことをChatで質問する
- what is your current job?
- do you have a patent?

※ patent が入力されるとピッグラテン語の応答になりコレは評価で失敗する。再実行時はピッグラテン語は指示されないので成功する。